In [1]:
%run Dataset_cleanup/cleanup_clinical.py

FeatherError: IO error: Unable to open file

In [ ]:
%run Dataset_cleanup/cleanup_gene_counts.py

## Benchmark Analysis of initial clinical presentation for  predictors of metastasis

In [ ]:
"""The Clinical data (including metastasis label) have a multitude of missing items."""
print("Total observations in original dataset:",clinical.shape[0])

not_labeled = y[y.isnull()]
y_labels = y[y.notnull()]

print("\nLabeled observations:",y_labels.shape[0],"\nUnlabeled observations removed:",not_labeled.shape[0])


In [ ]:
clinical = clinical.loc[y_labels.index]  #filter only observations where metastasis state is known
print("Filtering out missing metastasis state labels left",clinical.shape[0],"observations")

clinical.dropna(inplace = True) # Remove all observations where clinical data is missing
print("Removing NaN rows leaves ",clinical.shape[0]," observations in the feature set.") 

y_labels_NAdrop = y_labels.loc[clinical.index] #remove metastasis labels where no clinical data now exists due to NaN removal.

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.cross_validation import cross_val_score
from sklearn.metrics import recall_score
from sklearn.metrics import make_scorer, accuracy_score

In [ ]:
recaller = make_scorer(recall_score, greater_is_better=True, needs_proba=False, needs_threshold=False, pos_label='n1')
accuracy = make_scorer(accuracy_score, greater_is_better=True, needs_proba=False, needs_threshold=False)
clf = DecisionTreeClassifier(criterion='gini',
                            splitter='best',
                            max_depth=None,
                            min_samples_split=30,
                            min_samples_leaf=1,
                            min_weight_fraction_leaf=0.0,
                            max_features=None,
                            random_state=None,
                            max_leaf_nodes=None,
                            class_weight='balanced', #avoids the classifier being rewarded for choosing the most prevelant class each instance (n0)
                            presort=False)
recall_vals = cross_val_score(clf, clinical, y_labels_NAdrop, scoring=recaller, cv=5)
accuracy_vals = cross_val_score(clf, clinical, y_labels_NAdrop, scoring=accuracy, cv=5)
print('Recall_n1:  ',recall_vals.mean())

print('\nModel Accuracy score:  ',accuracy_vals.mean())
print("\nNull error rate for metastasis:  ",(sum(y_labels == 'n1') / len(y_labels)))

## Exploratory Analysis using Gene TPM counts as predictors of Metastasis

### Wrangle TPM counts DF

In [ ]:
print(gene_counts.index)

In [ ]:

print(gene_counts.shape)
print(len(y_labels.index))
X = gene_counts.loc[list(y_labels.index)]
print(X.shape)


In [ ]:
"""How can the length of X be larger than the index of y_labels which were used to select X?!  Must be duplicates, but
I thought index names had to be unique."""

seen = set()
uniq = []
for x in X.index:
    if x not in seen:
        uniq.append(x)
        seen.add(x)
    else:
        print(x)

### Feature Reduction

In [ ]:
from sklearn.feature_selection import SelectKBest, f_classif

In [ ]:
#selector = SelectKBest(f_classif, k = )